In [35]:
import sys
sys.path.append(r"/home/graham/Documents/Ironhack/Final-Project")
import src
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [36]:
path = ('/home/graham/Documents/Ironhack/Final-Project/data/combined_data.csv')
df = src.read_csv_file(path)


In [3]:
X = df[['2021 TR', '2022 TR', '2021 Inf', '2021 ED', '2022 ED', '2021 GHI', '2022 GHI']]
y = df['2022 Inf']

In [40]:
'''DECISION TREE'''
X = df[['2021 TR', '2022 TR', '2021 Inf', '2021 ED', '2022 ED', '2021 GHI', '2022 GHI']]
y = df['2022 Inf']

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Create pipeline with DecisionTreeRegressor
tree_reg = DecisionTreeRegressor(random_state=42)
pipeline = Pipeline(steps=[('regressor', tree_reg)])

# Define hyperparameters for grid search
parameters = {'regressor__max_depth': [3, 5, 7, 10],
              'regressor__min_samples_split': [2, 5, 10],
              'regressor__min_samples_leaf': [1, 2, 4],
              'regressor__max_features': ['sqrt', 'log2', None]}

# Create GridSearchCV object
grid_search = GridSearchCV(pipeline, parameters, cv=5, scoring='r2')

# Fit GridSearchCV object to training data
grid_search.fit(X_train, y_train)

# Print best hyperparameters and score
print("Best hyperparameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

# Use best model to predict on test data
y_pred = grid_search.predict(X_test)

# Evaluate performance of best model
r2_score = r2_score(y_test, y_pred)
print("Decision Tree R^2 score:", r2_score)

Best hyperparameters: {'regressor__max_depth': 5, 'regressor__max_features': 'sqrt', 'regressor__min_samples_leaf': 1, 'regressor__min_samples_split': 5}
Best score: -0.2551617791254649
Decision Tree R^2 score: -0.5942452712475599


In [37]:
'''LINEAR REGRESSION'''
X_train, X_test, y_train, y_test = train_test_split(
    df[['2021 TR', '2022 TR','2021 Inf', '2021 ED', '2022 ED', '2021 GHI', '2022 GHI']],
    df['2022 Inf'],
    test_size=0.2,
    random_state=42
)

scaler = StandardScaler()
lr = LinearRegression()

pipeline = Pipeline([('scaler', scaler), ('lr', lr)])

param_grid = {
    'lr__fit_intercept': [True, False],
    'lr__copy_X': [True, False]
}

grid = GridSearchCV(pipeline, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1)
grid.fit(X_train, y_train)

y_pred = grid.predict(X_test)


In [38]:
r2 = r2_score(y_test, y_pred)
print("Linear Regression R^2 score:", r2)
print("Best parameters:", grid.best_params_)
print("Best score:", grid.best_score_)



Linear Regression R^2 score: -0.8028201857368651
Best parameters: {'lr__copy_X': True, 'lr__fit_intercept': True}
Best score: -0.12324793371641292


In [39]:
'''Random Forest'''
# Define feature and target variables
X = df[['2021 TR', '2022 TR', '2021 Inf', '2021 ED', '2022 ED', '2021 GHI', '2022 GHI']]
y = df['2022 Inf']

# Scale the feature variables
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define parameter grid for grid search
param_grid = {
    'n_estimators': [100, 200, 500],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [3, 5, 7, None]
}

# Define random forest regressor
rf = RandomForestRegressor()

# Perform grid search to find the best hyperparameters
rf_grid = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1)
rf_grid.fit(X_train, y_train)

# Print best hyperparameters and score
print("Best parameters:", rf_grid.best_params_)
print("Best score:", rf_grid.best_score_)

# Predict on test set with best model from grid search
y_pred = rf_grid.predict(X_test)

# Evaluate model performance on test set
r2 = r2_score(y_test, y_pred)
print("Random Forest R^2 score:", r2)


Best parameters: {'max_depth': 5, 'max_features': 'log2', 'n_estimators': 100}
Best score: 0.4221601254785858
Random Forest R^2 score: -0.22820700709030017
